In [2]:
import keras
import numpy as np
from PIL import Image
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D
# from misc import load_data1, load_data2, load_data3, load_data4, load_data5, load_data6, load_data7, load_data8
from keras_vggface.vggface import VGGFace
from keras.models import Model, Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D

In [3]:
# data = load_data1()

K.set_learning_phase(1)

sess = K.get_session()
X = K.tf.placeholder(shape=(None, 1818, 1228, 3), dtype=K.tf.float32)
Y = K.tf.placeholder(shape=(None, 5), dtype=K.tf.float32)

In [4]:
vgg = VGGFace(include_top=False, input_tensor = X, input_shape=(1228, 1818, 3))
for layer in vgg.layers:
    layer.trainable = False

In [5]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1228, 1818, 3)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 1228, 1818, 64)    1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 1228, 1818, 64)    36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 614, 909, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 614, 909, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 614, 909, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 307, 454, 128)     0         
__________

In [6]:
p1 = Conv2D(filters=75, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=13).output)
p2 = Conv2D(filters=75, kernel_size=3, strides=1, padding='same')(vgg.get_layer(index=17).output)
p3 = Conv2D(filters=75, kernel_size=5, strides=1, padding='same')(MaxPool2D(strides=2)(p2))
p4 = Conv2D(filters=75, kernel_size=5, strides=1, padding='same')(MaxPool2D(strides=2)(p3))

p11 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p1))
p12 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p1)
p21 = K.sigmoid(Conv2D(filters=1, kernel_size=3, strides=1, padding='same')(p2))
p22 = Conv2D(filters=4, kernel_size=3, strides=1, padding='same')(p2)
p31 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p3))
p32 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(p3)
p41 = K.sigmoid(Conv2D(filters=1, kernel_size=5, strides=1, padding='same')(p4))
p42 = Conv2D(filters=4, kernel_size=5, strides=1, padding='same')(p4)

p12 = K.concatenate(tensors=[p11, p12], axis = -1)
p22 = K.concatenate(tensors=[p21, p22], axis = -1)
p32 = K.concatenate(tensors=[p31, p32], axis = -1)
p42 = K.concatenate(tensors=[p41, p42], axis = -1)

In [54]:
pred = [K.reshape(p12, (-1, 5)), K.reshape(p22, (-1, 5)), K.reshape(p32, (-1, 5)), K.reshape(p42, (-1, 5))]

In [69]:
pred = K.concatenate(pred, axis = 0)

In [118]:
def NMS(boxes = pred):
    
    Z0 = K.tf.image.non_max_suppression(boxes=K.reshape(boxes[:, 1:], (-1, 4)), scores=K.reshape(boxes[:, 0], (-1, )),
                                        max_output_size=25, iou_threshold=0.55)
    A = K.reshape(K.tf.gather_nd(boxes, Z0), (-1, 5))
    return A

In [112]:
def sigmoid(X):
    return 1/(1 + K.exp(-X))

def relu(X):
    return K.maximum(X, 0)

def loss(Y, y, a=0.8):
    if K.tf.size(Y) != K.tf.size(y):
        n, _ = Y.get_shape().as_list()
        y_ = y[:n]
        
    softMax = K.mean(-(Y[:, 0] * K.log(y_[:, 0]) + (1 - Y[:, 0]) * K.log(1 - y_[:, 0])))
    mse = K.mean(K.square(Y[:, 1:] - y_[:, 1:]))
    return 1.15 * softMax + 0.8 * mse

In [119]:
sess.run(K.tf.global_variables_initializer())

In [120]:
b = sess.run([NMS()], feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg")), axis = 0)})[0]
bbox = NMS()

InvalidArgumentError: index innermost dimension length must be <= params rank; saw: 25 vs. 2
	 [[Node: GatherNd_18 = GatherNd[Tindices=DT_INT32, Tparams=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](concat_7, non_max_suppression_28/NonMaxSuppressionV2/_333)]]
	 [[Node: Reshape_86/_335 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_282_Reshape_86", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'GatherNd_18', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/chawat/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/chawat/.local/lib/python3.

In [110]:
b

array([[ 0.99861634, -3.801135  , -1.5769863 ,  1.8805543 , -0.11746165],
       [ 0.99832517, -3.174851  , -1.205837  ,  2.4337816 , -0.25086924],
       [ 0.9983209 , -3.91178   , -2.3245049 ,  2.5524526 , -0.5206986 ],
       [ 0.9982597 , -3.024315  , -1.3457122 ,  2.021164  , -0.6230618 ],
       [ 0.99815804, -3.363327  , -2.8222976 ,  3.390693  , -0.8742572 ],
       [ 0.99805486, -2.7460709 , -1.5610839 ,  2.145849  , -0.826748  ],
       [ 0.9979607 , -3.1729515 , -1.883345  ,  1.4105705 , -0.87786925],
       [ 0.99785256, -3.5335262 , -1.9670486 ,  2.6487415 ,  0.30191952],
       [ 0.9977252 , -4.24036   , -2.5700297 ,  2.3681843 , -1.4868052 ],
       [ 0.99756396, -4.350228  , -3.1276505 ,  3.549464  , -0.03473406],
       [ 0.997451  , -2.7215564 , -1.0699437 ,  2.1070044 ,  0.16294491],
       [ 0.99707067, -2.9855404 , -1.9729292 ,  1.7804163 , -1.3280082 ],
       [ 0.99694175, -2.5283003 , -1.7363348 ,  0.20214981, -0.6705291 ],
       [ 0.9969097 , -1.1688659 , -1.1

In [116]:
Loss = loss(Y, bbox)

In [117]:
opt = K.tf.train.AdagradOptimizer(learning_rate=0.001)
train_ = opt.minimize(Loss)
sess.run(K.tf.global_variables_initializer())

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [10]:
def train(epochs = 10):
    for e in range(epochs):
        cost = 0
#         for i, d in enumerate(data):
        _, L = sess.run([train_, Loss], feed_dict={X: np.expand_dims(np.array(Image.open("x.jpg")), axis=0), Y: np.ones((2, 5)).reshape((-1, 5))})
        cost += L
        if e % 100 == 0:
            print("epoch {}, iter {} , loss {}".format(e, i , cost/len(data)))

In [ ]:
train(epochs = 12)